# ClusterEnsemble demonstration
Some example usage of how to build up a dataframe of galaxy cluster properties, including NFW halo profiles. Each cluster is treated as an individual, meaning we track its individual mass and redshift, and other properties. This is useful for fitting a stacked weak lensing profile, for example, where you want to avoid fitting a single average cluster mass.

In [ ]:
from __future__ import absolute_import, division, print_function

%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
from astropy import units
from matplotlib import pyplot as plt

#import seaborn as sns; sns.set() 

In [ ]:
from clusters import ClusterEnsemble

### Create a ClusterEnsemble object by passing in a numpy array (or list) of redshifts

In [ ]:
z = np.array([0.1,0.2,0.3])
c = ClusterEnsemble(z)
c.describe

### Display what we have so far
Below the DataFrame (which so far only contains the cluster redshifts), we see the default assumptions for the power-law slope and normalization that will be used to convert richness $N_{200}$ to mass $M_{200}$. We'll see how to change those parameters below.

In [ ]:
c.show()

### Add richness values to the dataframe
This step will also generate $M_{200}$, $r_{200}$, $c_{200}$, scale radius $r_s$, and other parameters, assuming the scaling relation given below.

In [ ]:
n200 = np.ones(3)*20.
c.n200 = n200
c.show()

### Access any column of the dataframe as an array
Notice that [astropy units](http://docs.astropy.org/en/stable/units/) are present for the appropriate columns.

In [ ]:
print('z: \t', c.z)
print('n200: \t', c.n200)
print('r200: \t', c.r200)
print('m200: \t', c.m200)
print('c200: \t', c.c200)
print('rs: \t', c.rs)

### If you don't want units, you can get just the values

In [ ]:
c.r200.value

### Or access the Pandas DataFrame directly

In [ ]:
c.dataframe

### Change the redshifts or richness values
These changes will propogate to all redshift-dependant or richness-dependant cluster attributes, as appropriate.

In [ ]:
c.z = np.array([0.4,0.5,0.6])
c.dataframe

In [ ]:
c.n200 = [20,30,40]
c.show()

### Change the parameters in the mass-richness relation
Either or both of the keyword parameters "slope" and "norm" can be passed to the update_massrichrelation() method.

In [ ]:
c.update_massrichrelation(slope = 1.5)
c.show()

### Show basic table
Perhaps we don't want the fancy pandas formatting on our table, or maybe we're not working in the Jupyter notebook.

In [ ]:
c.show(notebook = False)

## Calculate $\Sigma(r)$ and $\Delta\Sigma(r)$ for NFW model
First select the radial bins in units of Mpc.

In [ ]:
rmin, rmax = 0.1, 5. #Mpc
nbins = 50
rbins = np.logspace(np.log10(rmin), np.log10(rmax), num = nbins)
#rbins

In [ ]:
time_c = %timeit -o c.calc_nfw(rbins)
sigma = c.sigma_nfw
deltasigma = c.deltasigma_nfw

In [ ]:
sigma

In [ ]:
fig = plt.figure(figsize=(12,5))
fig.suptitle('Centered NFW Cluster Profiles', size=30)

first = fig.add_subplot(1,2,1)
second = fig.add_subplot(1,2,2)

for rich, profile in zip(c.n200,deltasigma):
    first.plot(rbins, profile, label='$N_{200}=$ '+str(rich))
first.set_xscale('log')
first.set_xlabel('$r\ [\mathrm{Mpc}]$', fontsize=20)
first.set_ylabel('$\Delta\Sigma(r)\ [\mathrm{M}_\mathrm{sun}/\mathrm{pc}^2]$', 
                 fontsize=20)
first.set_xlim(rbins.min(), rbins.max())
first.legend(fontsize=20)


for rich, profile in zip(c.n200,sigma):
    second.plot(rbins, profile, label='$N_{200}=$ '+str(rich))
second.set_xscale('log')
second.set_xlabel('$r\ [\mathrm{Mpc}]$', fontsize=20)
second.set_ylabel('$\Sigma(r)\ [\mathrm{M}_\mathrm{sun}/\mathrm{pc}^2]$', 
                 fontsize=20)
second.set_xlim(rbins.min(), rbins.max())
second.legend(fontsize=20)


fig.tight_layout()
plt.subplots_adjust(top=0.88)

# Calculate Miscentered NFW Profiles
First select the offsets in units of Mpc. The offset values parameterize the width of the Gaussian distribution of offsets, and is $\sigma_\mathrm{off}$ in Equation 11 of [Ford et al 2015](http://arxiv.org/abs/1409.3571).

In [ ]:
offsets = np.array([0.09,0.09,0.09])
time_c_offset = %timeit -o c.calc_nfw(rbins, offsets=offsets)
deltasigma_off = c.deltasigma_nfw
sigma_off = c.sigma_nfw

In [ ]:
fig = plt.figure(figsize=(12,5))
fig.suptitle('Miscentered NFW Cluster Profiles', size=30)

first = fig.add_subplot(1,2,1)
second = fig.add_subplot(1,2,2)

for rich, profile in zip(c.n200,deltasigma_off):
    first.plot(rbins, profile, label='$N_{200}=$ '+str(rich))
first.set_xscale('log')
first.set_xlabel('$r\ [\mathrm{Mpc}]$', fontsize=20)
first.set_ylabel('$\Delta\Sigma^\mathrm{sm}(r)\ [\mathrm{M}_\mathrm{sun}/\mathrm{pc}^2]$', fontsize=20)
first.set_xlim(rbins.min(), rbins.max())
first.legend(fontsize=20)


for rich, profile in zip(c.n200,sigma_off):
    second.plot(rbins, profile, label='$N_{200}=$ '+str(rich))
second.set_xscale('log')
second.set_xlabel('$r\ [\mathrm{Mpc}]$', fontsize=20)
second.set_ylabel('$\Sigma^\mathrm{sm}(r)\ [\mathrm{M}_\mathrm{sun}/\mathrm{pc}^2]$', 
                 fontsize=20)
second.set_xlim(rbins.min(), rbins.max())
second.legend(fontsize=20)


fig.tight_layout()
plt.subplots_adjust(top=0.88)

## NEW: a Python implentation of the NFW calculations 
Set the keyword parameter "use_c = False" to use python only. Note that this method is significantly faster than the C code, for the perfectly centered case (because the C version suboptimally writes/reads to disc), but far slower for the miscentered case (which includes calculating a double integral). *Currently, the Python option calculates centered $\Sigma(r)$ and $\Delta\Sigma(r)$ profiles, and the miscentered profile $\Sigma^\mathrm{sm}(r)$, but not yet $\Delta\Sigma^\mathrm{sm}(r)$...*

In [ ]:
time_py = %timeit -o c.calc_nfw(rbins, use_c = False)
sigma_py = c.sigma_nfw
dsigma_py = c.deltasigma_nfw

In [ ]:
sigma_py = c.sigma_nfw
dsigma_py = c.deltasigma_nfw

In [ ]:
#check the results match
np.testing.assert_allclose(sigma_py, sigma, rtol = 10**-4)
np.testing.assert_allclose(dsigma_py, deltasigma, rtol = 10**-4)

### Python Calculation of $\Sigma^\mathrm{sm}(r)$
Even with dblquad and large epsabs, epsrel allowances, this is way slow.

In [ ]:
time_py_offset_justsigma = %timeit -o c.calc_nfw(rbins, offsets = offsets, use_c = False, \
                                                 epsabs = 0.5, epsrel = 0.5)

In [ ]:
sigma_off_py = c.sigma_nfw
#deltasigma_off_py = c.deltasigma_nfw #not yet implemented

In [ ]:
sigma_off_py

In [ ]:
#check the results match... 
np.testing.assert_allclose(sigma_off_py, sigma_off, rtol = 10**-1)

#looks like 10**-1 to 10**-2 is the level at which the offset 
#profiles match, depending on what I set the epsabs, epsrel to be.

### Timing Comparisons: Python vs C
- For centered halos, Python implementation is about 30% faster than C.
- For offset halos, need $\Delta\Sigma^\mathrm{sm}(r)$ implemented in Python for full comparison.

In [ ]:
#Centered: Python vs C 
(time_c.best - time_py.best) / time_c.best

In [ ]:
#Miscentered: Python vs C 
#(time_c_off - time_py_off) / time_c_off

### To Do: 
- fix bug sometimes giving Inf in first bin of smoothed profiles
- replace smd_nfw.c with cython version
- option to pass in a $M_{prelim}$ and $M_{200} = a \times$ $M_{prelim}$ relation?